In [ ]:
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import requests
from datetime import datetime
import string
import json
import xml.etree.ElementTree as ET
import httpx
from lxml import html
import nltk
import re

nltk.download("stopwords")

In [ ]:
# Opening config file -> this config file is found in a config folder, the config structure is:
# {access_token_zenodo: "......"
# openai_api_key: "......"}

config = open('config/config.json', 'r')
config = json.load(config)

In [ ]:
# possible to use OAI-PMH (Open Archives Initiative Protocol for Metadata Harvesting) to extract zenodo and other data
# however, see no option to select for 'soil', this can be fed as parameter when using zenodo API directly, so worked further with Zenoodo API
base_url = "https://zenodo.org/oai2d"


In [ ]:
# Define the API endpoint URL
api_url = "https://zenodo.org/api/records"

# query for datasources with 'soil'
# get first 200 datasources
# loop through pages to retrieve next part of data
# only get sources with status published
#  options: https://developers.zenodo.org/?python#list36

# list to gather all data through API requests
zenodo_data = []

# loop through pages to retreive next part of data
for i in range(1,11):
    print(f'loop num {i}')
    params = {'q': 'soil',
              'size':'20',
              'page':f'{i}',
              'status':'published',
              'access_token': config['access_token_zenodo']}
    headers = {"Content-Type": "application/json"}

    # Make the API request
    response = requests.get(api_url,params = params, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        zenodo_data = zenodo_data + data['hits']['hits']
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    
zenodo_data[0]

In [ ]:
# gather the id's in list to then extract data directly as dublincore representation
id_list = []
for i in range(0,len(zenodo_data)):
    id_list.append(zenodo_data[i]['id'])

print(f'length of list is {len(id_list)}, first 10 ids are: {id_list[:10]}')

In [ ]:
#NLP functions

# Generate a set of stop words to remove from text.
stoplist_base = """ the is and are of"""
stoplist_symbols = "km +/- _ … -"

stopwords = {
        w
        for stoplist in [
            stoplist_base.split(),
            stoplist_symbols.split(),
            nltk.corpus.stopwords.words("english"),
        ]
        for w in stoplist
        if w  # "if w" makes that no empty strings ('') are added
    }

# Genereate patterns for cleaning up text
tags_websites_emails_pattern = re.compile(r"<[^>]+>|\n\n|\n|\r|http\S+|www\S+|\S+@\S+|/")
punctuation_pattern = re.compile(f"[{string.punctuation}]")
numbers_pattern = re.compile(r"\b\d+(?:\.\d+)?\s+")
time_pattern = re.compile(r"(?:\d+[uh:](?:\d\d+)?)\b")

def clean_normalize(row):
    """Clean and normalize the text data."""
    row = row.lower()
    row = re.sub(tags_websites_emails_pattern, " ", row)
    row = re.sub(punctuation_pattern, " ", row)
    row = re.sub(numbers_pattern, " ", row)  # replace numbers
    row = re.sub(time_pattern, " ", row)  # replace time indications
    row = re.sub(r"(\d+\S*)\b", " ", row)  # drop all numbers with optional word
    row = re.sub(r"\W", " ", row, flags=re.I)  # remove all the non-word characters
    row = re.sub(r"\s+", " ", row).strip()  # replace multiple white strings
    return row


In [ ]:
# extract DublinCore standardized metadata and generate json with all data
zenodo_dc_data = {}
for i in range(len(id_list)):
    # print(i)

    # Define the API endpoint URL
    api_url = f"https://zenodo.org/records/{id_list[i]}/export/dublincore"
    params = {'access_token': config['access_token_zenodo']}
    headers = {"Content-Type": "application/json"}

    # Make the API request
    response = requests.get(api_url,params = params, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        try:
            # Parse the XML response
            root = ET.fromstring(response.content)

            # Check if the root has child elements
            if root:
                # Find the dc:identifier element
                element = root.find('.//{http://purl.org/dc/elements/1.1/}identifier')
                identifier_element = element.text if element is not None else None

                # Find the dc:title element and clean & normalize the title with NLP functions
                element = root.find('.//{http://purl.org/dc/elements/1.1/}title')
                title_element = [x.lower() for x in clean_normalize(element.text).split()] if element is not None else None
                title_element = [token for token in title_element if token not in stopwords] if title_element is not None else None

                 # Find the dc:date element
                element = root.find('.//{http://purl.org/dc/elements/1.1/}date')
                date_element = element.text if element is not None else None

                 # Find the dc:description element
                element = root.find('.//{http://purl.org/dc/elements/1.1/}description')
                description_element = element.text if element is not None else None

                # Find the dc:creator element
                element = root.find('.//{http://purl.org/dc/elements/1.1/}creator')
                creator_element = [x.lower() for x in re.sub(r"'", "", element.text).split(',')] if element is not None else None

                # Find the dc:subject element
                element = root.find('.//{http://purl.org/dc/elements/1.1/}subject')
                subject_element = [x.lower() for x in re.sub(r"'", "", element.text).split(',')] if element is not None else None

                # Find the dc:type element
                element = root.find('.//{http://purl.org/dc/elements/1.1/}type')
                type_element = element.text if element is not None else None

                # combine to dict
                zenodo_dict={
                    i:{
                        "dc:identifier": identifier_element,
                        "dc:title": title_element,
                        "dc:date": date_element,
                        "dc:description": description_element,
                        "dc:creator": creator_element,
                        "dc:subject": subject_element,
                        "dc:type": type_element
                    }
                }
                
                #add to general json
                zenodo_dc_data.update(zenodo_dict)

            else:
                print("No repositories found in the response.")
        except ET.ParseError:
            print("Error: Unable to parse XML content.")
    else:
        # Print an error message if the request was not successful
        print(f"Error: Unable to fetch data. Status code: {response.status_code}")
        print(response.text)

    
zenodo_dc_data[0]

In [ ]:
# test, see title of certain datasource
zenodo_data[0]['metadata']['title']

In [ ]:
# Connect to local neo4J db
neo4j_config = {
    'pathway': '.....',
    'username': '.....',
    'password': '.....',
}

pathway = neo4j_config['pathway']
username = neo4j_config['username']
password = neo4j_config['password']

driver = GraphDatabase.driver(pathway, auth=(username, password))
graph = Graph(pathway, auth=(username, password))  # Replace with your database credentials
gds = GraphDataScience(pathway, auth=(username, password))

In [ ]:
# drop existing data in Neo4J graph
graph.run("MATCH (n) DETACH DELETE n") #delete database

In [ ]:
# start converting the data to knowledge graph and inport into Neo4J database
# import info for each row
def create_nodes_and_relationships(tx, row):
    # Define Cypher query to create nodes and relationships
    query = (
        "MERGE (node1:SoilwiseDataSource {identifier: $value1})"
        "MERGE (node2:PublicationDate {publicationDate: $value2})"
        "MERGE (node3:DataSourceType {dataSourceType: $value3})"
       
        "CREATE (node1)-[:PUBLISHED_ON]->(node2)" 
        "CREATE (node1)-[:TYPE_OF_DATASOURCE]->(node3)"
    )
    # Update the query with DataFrame data
    parameters = {
        "value1": row['dc:identifier'],
        "value2": row['dc:date'],
        "value3": re.sub(r"info:eu-repo/semantics/", "", row['dc:type']) 
    }
    # Execute the query
    tx.run(query, **parameters)
    
with driver.session() as session:
    for row in range(len(zenodo_dc_data)):
        session.execute_write(create_nodes_and_relationships, zenodo_dc_data[row])
session.close()


def write_creator_nodes_and_relations(tx, row):
        node_num = 2
        # import general info
        query = ("MERGE (node1:SoilwiseDataSource {identifier: $value1})" )
        # Update the query with  DataFrame data
        parameters = { "value1": row['dc:identifier'],}
        for i in row['dc:creator']:
            if i != '':
                i = i.strip()
                sub_query = (
                    f"MERGE (node{node_num}:Creator {{CreatorName: '{i}'}}) "
                    f"MERGE (node1)-[:IS_CREATED_BY]->(node{node_num}) "
                )
                query += sub_query
                node_num += 1  
        # Execute the query
        tx.run(query, **parameters)


with driver.session() as session:
    for row in range(len(zenodo_dc_data)):
        if zenodo_dc_data[row]['dc:creator']:
            session.execute_write(write_creator_nodes_and_relations, zenodo_dc_data[row])
session.close()



def write_subject_nodes_and_relations(tx, row):
        node_num = 2
        # import general info
        query = ("MERGE (node1:SoilwiseDataSource {identifier: $value1})" )
        # Update the query with  DataFrame data
        parameters = { "value1": row['dc:identifier'],}
        for i in row['dc:subject']:
            if i != '':
                i = i.strip()
                sub_query = (
                    f"MERGE (node{node_num}:Subject {{SubjectName: '{i}'}}) "
                    f"MERGE (node1)-[:HAS_SUBJECT]->(node{node_num}) "
                )
                query += sub_query
                node_num += 1  
        # Execute the query
        tx.run(query, **parameters)


with driver.session() as session:
    for row in range(len(zenodo_dc_data)):
        if zenodo_dc_data[row]['dc:subject']:
            session.execute_write(write_subject_nodes_and_relations, zenodo_dc_data[row])
session.close()


def write_title_nodes_and_relations(tx, row):
        node_num = 2
        # import general info
        query = ("MERGE (node1:SoilwiseDataSource {identifier: $value1})" )
        # Update the query with  DataFrame data
        parameters = { "value1": row['dc:identifier'],}
        for i in row['dc:title']:
            if i != '':
                i = i.strip()
                sub_query = (
                    f"MERGE (node{node_num}:Title {{TitleName: '{i}'}}) "
                    f"MERGE (node1)-[:HAS_TITLE]->(node{node_num}) "
                )
                query += sub_query
                node_num += 1  
        # Execute the query
        tx.run(query, **parameters)


with driver.session() as session:
    for row in range(len(zenodo_dc_data)):
        if zenodo_dc_data[row]['dc:title']:
            session.execute_write(write_title_nodes_and_relations, zenodo_dc_data[row])
session.close()


# mossing values were filled in with "" or 0, drop these
graph.run("""
MATCH (n)
where n.SubjectName = ""
DETACH DELETE n
""")

In [ ]:
# setup LLM to consult knowledge in NEO4J
import os

from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

os.environ['OPENAI_API_KEY'] = config['openai_api_key']

graph = Neo4jGraph(
    url=neo4j_config['pathway'], 
    username=neo4j_config['username'], 
    password=neo4j_config['password'])

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [ ]:
# ask queston to knowledge graph

chain.run("""
Which datasources deal with scotland?

When querying for certain subject always use lowercase.
Generate a short answer using all retreived results.
""")

In [ ]:
# ask queston to knowledge graph

chain.run("""
Which datasources have a title about scotland?

When querying for certain subject always use lowercase.
Generate a short answer using all retreived results
""")

In [ ]:
# ask queston to knowledge graph

chain.run("""
Which datasources have a subject 'land use'?

When querying for certain subject always use lowercase.
Generate a short answer using all retreived results
""")

In [ ]:
# ask queston to knowledge graph

chain.run("""
Which datasources have a subject 'cover crops'?

When querying for certain subject always use lowercase.
Generate a short answer using all retreived results
""")

In [ ]:
# ask queston to knowledge graph

chain.run("""
When was a report published?

When querying for certain subject always use lowercase.
Generate a short answer using all retreived results
""")

In [ ]:
# ask queston to knowledge graph

chain.run("""
When was a report published?

Take into account that the question 
"When was a report published?"
should be interpreted as the cypher query:
MATCH (s:SoilwiseDataSource)-[:PUBLISHED_ON]->(p:PublicationDate)
WHERE (s)-[:TYPE_OF_DATASOURCE]->(:DataSourceType {dataSourceType: 'report'})
RETURN p

When querying for certain subject always use lowercase.
Generate a short answer using all retreived results
""")

In [ ]:
# ask queston to knowledge graph

chain.run("""
When was an article published?

Take into account that the question 
"When was a report published?"
should be interpreted as the cypher query:
MATCH (s:SoilwiseDataSource)-[:PUBLISHED_ON]->(p:PublicationDate)
WHERE (s)-[:TYPE_OF_DATASOURCE]->(:DataSourceType {dataSourceType: 'report'})
RETURN p

The following are a datasource when generating the cypher query: article, report, poster, software, other
When querying for certain subject always use lowercase.
Generate a short answer using all retreived results
""")

In [ ]:
# check which graph projections exist
gds.graph.list()

In [ ]:
# if necessary, clean up existing projection before making new one
G = gds.graph.get('creators_centrality')
gds.graph.drop(G)

In [ ]:
# if necessary, clean up existing projection before making new one
G = gds.graph.get('title_similarity')
gds.graph.drop(G)

In [ ]:
# if necessary, clean up existing projection before making new one
G = gds.graph.get('keyword_similarity')
gds.graph.drop(G)

In [ ]:
# create graph projection we are interested in
G, result = gds.graph.project(
    "creators_centrality",  # Graph name
    ["Creator", "SoilwiseDataSource"],  # Node projection
    {"IS_CREATED_BY": {"orientation": "UNDIRECTED"}},  # Relationship projection
    readConcurrency=4  # Configuration parameters
)

# stream predictions using "degree centrality" algorithm
stream_result = gds.degree.stream(G)
data_nodes = gds.run_cypher(
        """
          MATCH (n:Creator)
          RETURN id(n) as id, n.CreatorName as CreatorName
        """
    )

# merge predictions and original data to report
data = pd.merge(stream_result, data_nodes, how="left", left_on="nodeId", right_on="id")
data = data.dropna()
data = data.sort_values('score', ascending=False)
data = data[["CreatorName", "score"]]

data

In [ ]:
# # create graph projection we are interested in
G, result = gds.graph.project(
    "title_similarity",  # Graph name
    [ "SoilwiseDataSource", "Title"],  # Node projection
    {"HAS_TITLE": {"orientation": "UNDIRECTED"}},  # Relationship projection
    readConcurrency=4  # Configuration parameters
)

# stream predictions using "node similarity" algorithm
stream_result = gds.nodeSimilarity.stream(G)
data_nodes = gds.run_cypher(
        """
          MATCH (n:SoilwiseDataSource)
          RETURN id(n) as id, n.identifier as SoilwiseDataSource_id
        """
    )

# merge predictions and original data to report
# merge datasets and rename
data = pd.merge(stream_result, data_nodes, how="left", left_on="node1", right_on="id")
data = pd.merge(data, data_nodes, how="left", left_on="node2", right_on="id")
data = data.rename(columns={"SoilwiseDataSource_id_x": "SoilwiseDataSource_id1", "SoilwiseDataSource_id_y": "SoilwiseDataSource_id2"})
data = data[["SoilwiseDataSource_id1", "SoilwiseDataSource_id2", "similarity"]]

# make dataframe with distinct title words for each datasource
data_titles = gds.run_cypher(
        """
            MATCH (n:SoilwiseDataSource) -[:HAS_TITLE]-> (t:Title)
            RETURN n.identifier AS id, COLLECT(DISTINCT t.TitleName) AS distinct_titles
        """
    )
data_titles.distinct_titles = data_titles.distinct_titles.apply(lambda x: list(set(x)))

# add title words
data = pd.merge(data, data_titles, how="left", left_on="SoilwiseDataSource_id1", right_on="id")
data = pd.merge(data, data_titles, how="left", left_on="SoilwiseDataSource_id2", right_on="id")

# cleanup and select columns to report
data = data.dropna()
data = data.sort_values('similarity', ascending=False)
data = data[["SoilwiseDataSource_id1", "SoilwiseDataSource_id2", "similarity","distinct_titles_x", "distinct_titles_y"]]
data = data.rename(columns={"distinct_titles_x": "distinct_titles_1", "distinct_titles_y": "distinct_titles_2"})
data = data.drop_duplicates(subset=['SoilwiseDataSource_id1', 'SoilwiseDataSource_id2','similarity'])

# Create a new column with frozensets of the sorted pairs
data['pair_frozenset'] = data.apply(lambda row: frozenset(sorted([row['SoilwiseDataSource_id1'], row['SoilwiseDataSource_id2']])), axis=1)

# Drop duplicates based on the frozensets
data = data.drop_duplicates(subset='pair_frozenset')

# Resetting the index after dropping duplicates
data = data.drop(columns='pair_frozenset').reset_index(drop=True)

data.head(20)

In [ ]:
# create graph projection we are interested in
G, result = gds.graph.project(
    "keyword_similarity",  # Graph name
    [ "SoilwiseDataSource", "Subject"],  # Node projection
    {"HAS_SUBJECT": {"orientation": "UNDIRECTED"}},  # Relationship projection
    readConcurrency=4  # Configuration parameters
)

# stream predictions using "node similarity" algorithm
stream_result = gds.nodeSimilarity.stream(G)
data_nodes = gds.run_cypher(
        """
          MATCH (n:SoilwiseDataSource)
          RETURN id(n) as id, n.identifier as SoilwiseDataSource_id
        """
    )

# merge predictions and original data to report
# merge datasets and rename
data = pd.merge(stream_result, data_nodes, how="left", left_on="node1", right_on="id")
data = pd.merge(data, data_nodes, how="left", left_on="node2", right_on="id")
data = data.rename(columns={"SoilwiseDataSource_id_x": "SoilwiseDataSource_id1", "SoilwiseDataSource_id_y": "SoilwiseDataSource_id2"})
data = data[["SoilwiseDataSource_id1", "SoilwiseDataSource_id2", "similarity"]]

# make dataframe with distinct title words for each datasource
data_subjects = gds.run_cypher(
        """
            MATCH (n:SoilwiseDataSource) -[:HAS_SUBJECT]-> (t:Subject)
            RETURN n.identifier AS id, COLLECT(DISTINCT t.SubjectName) AS distinct_subjects
        """
    )
data_subjects.distinct_subjects = data_subjects.distinct_subjects.apply(lambda x: list(set(x)))

# add title words
data = pd.merge(data, data_subjects, how="left", left_on="SoilwiseDataSource_id1", right_on="id")
data = pd.merge(data, data_subjects, how="left", left_on="SoilwiseDataSource_id2", right_on="id")

# cleanup and select columns to report
data = data.dropna()
data = data.sort_values('similarity', ascending=False)
data = data[["SoilwiseDataSource_id1", "SoilwiseDataSource_id2", "similarity","distinct_subjects_x", "distinct_subjects_y"]]
data = data.rename(columns={"distinct_subjects_x": "distinct_subjects_1", "distinct_subjects_y": "distinct_subjects_2"})
data = data.drop_duplicates(subset=['SoilwiseDataSource_id1', 'SoilwiseDataSource_id2','similarity'])

# Create a new column with frozensets of the sorted pairs
data['pair_frozenset'] = data.apply(lambda row: frozenset(sorted([row['SoilwiseDataSource_id1'], row['SoilwiseDataSource_id2']])), axis=1)

# Drop duplicates based on the frozensets
data = data.drop_duplicates(subset='pair_frozenset')

# Resetting the index after dropping duplicates
data = data.drop(columns='pair_frozenset').reset_index(drop=True)

data.head(100)